# 200 Pozos petroleros nuevos

## Descripción del proyecto

Trabajas en la compañía de extracción de petróleo OilyGiant. Tu tarea es encontrar los mejores lugares donde abrir 200 pozos nuevos de petróleo.

Condiciones:

* Solo se debe usar la regresión lineal para el entrenamiento del modelo.
* Al explorar la región, se lleva a cabo un estudio de 500 puntos con la selección de los mejores 200 puntos para el cálculo del beneficio.
* El presupuesto para el desarrollo de 200 pozos petroleros es de 100 millones de dólares.
* Un barril de materias primas genera 4.5 USD de ingresos. El ingreso de una unidad de producto es de 4500 dólares (el volumen de reservas está expresado en miles de barriles).
* Después de la evaluación de riesgo, mantén solo las regiones con riesgo de pérdidas inferior al 2.5%. De las que se ajustan a los criterios, se debe seleccionar la región con el beneficio promedio más alto.

## 1. Preparación de los datos 

Se hace la importación de los datos y se mostrará la información preliminar usando `info()`

In [1]:
# Importación de librerías
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [2]:
# Se cargan los datos
data_1 = pd.read_csv('/datasets/geo_data_0.csv')
data_2 = pd.read_csv('/datasets/geo_data_1.csv')
data_3 = pd.read_csv('/datasets/geo_data_2.csv')

# Se muestra la información de los tres datasets
data_1.info()
print()
data_2.info()
print()
data_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Nul

Se mostrarán las características estadísticas de los datasets para ver si los datos difieren mucho unos de otros y ver en un escalado de características es necesario

In [3]:
# Caracterísitcas estadísticas
print(data_1.describe())
print()
print(data_2.describe())
print()
print(data_3.describe())

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%      

Se imprimiran 3 muestras de cada dataset para corroborar que los tipos de datos sean correctos

In [4]:
display(data_1.head(3))
print()
display(data_2.head(3))
print()
display(data_3.head(3))
print()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910


Conclusiones:

* No hay valores ausentes y podemos observar que los 3 datasets contienen 100000 líneas cada uno y 5 columnas. Los tipos de datos son correctos.
* Es necesario un escalado de características porque los datos difieren mucho unos de otros.
* Nuestro target es la columna `product` porque es la columna que contiene el volumen de reservas en el pozo (miles de barriles)
* Se borrará la columna `id` de los datasets para entrenar el modelo. Se toma esta descición porque el id de cada pozo no afecta para predecir si el pozo es redituable, sólo es un identificador.

Se borra la columna `id`

In [5]:
data_1 = data_1.drop(['id'], axis=1)
data_2 = data_2.drop(['id'], axis=1)
data_3 = data_3.drop(['id'], axis=1)

# Verificación de que ya no existe la columna id
display(data_1.head(5))
display(data_2.head(5))
display(data_3.head(5))

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


,f0,f1,f2,product
0,-15.001348,-8.276000,-0.005876,3.179103
1,14.272088,-3.475083,0.999183,26.953261
2,6.263187,-5.948386,5.001160,134.766305
3,-13.081196,-11.506057,4.999415,137.945408
4,12.702195,-8.147433,5.004363,134.766305


,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673
1,0.262778,0.269839,-2.530187,56.069697
2,0.194587,0.289035,-5.586433,62.871910
3,2.236060,-0.553760,0.930038,114.572842
4,-0.515993,1.716266,5.899011,149.600746


Escalado de datos, la columna `product` no se va a escalar porque es la variable objetivo

## 2. Entrenamiento del modelo de regresión lineal

Para el entrenamiento del modelo, se creara una función que se encargará de realizar dicho entrenamiento, así se podrá llamar para cada dataset que se tiene y se evitará escribirlo para cada uno, haciendolo más eficiente

Antes de la función de entrenamiento, se hará una función diferente para la divisón del conjunto `features` (características) y el conjunto `target` (objetivo) en una proporción de 75:25. De igual manera en dicha función se realizará el escalado de los datos.

Esta función va a devolver `features_train`, `features_valid`, `target_train`, `target_valid`.

In [6]:
def split_data(data):
    # Separa los conjuntos de entrenamiento y validacion
    features = data.drop(['product'], axis=1)
    target = data['product']
    
    # Separa los conjuntos de features_train, features_valid, target_train, target_valid
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
    
    # Columnas que se van a escalar
    numeric = ['f0', 'f1', 'f2']
    # Escalado de las características
    scaler = StandardScaler()
    scaler.fit(features_train[numeric])
    features_train[numeric] = scaler.transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric])
    
    return features_train, features_valid, target_train, target_valid

In [7]:
# Llamada a la función para cada dataset
features_train_1, features_valid_1, target_train_1, target_valid_1 = split_data(data_1)
features_train_2, features_valid_2, target_train_2, target_valid_2 = split_data(data_2)
features_train_3, features_valid_3, target_train_3, target_valid_3 = split_data(data_3)

/tmp/ipykernel_71/591743370.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_train[numeric] = scaler.transform(features_train[numeric])
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

Ahora se va a crear la función para entrenar el modelo, la cual devuelve las predicciones (`predicted_valid`) y las respuestas correctas (`target_valid`) del conjunto de validación. Las predicciones serán cnvertidads a un objeto `Series` por comodidad al manejar las predicciones.

In [8]:
# Función para entrenar los datasets con el modelo
def train_linear_model(features_train, features_valid, target_train, target_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    predicted_valid = pd.Series(predicted_valid)
    return target_valid, predicted_valid

In [9]:
# Llamada a la función, guardando predicted_value y target en variables
target_valid_1, predicted_valid_1 = train_linear_model(features_train_1, features_valid_1, target_train_1, target_valid_1)
target_valid_2, predicted_valid_2 = train_linear_model(features_train_2, features_valid_2, target_train_2, target_valid_2)
target_valid_3, predicted_valid_3 = train_linear_model(features_train_3, features_valid_3, target_train_3, target_valid_3)

Para asegurarnos que los targets y las predicciones tengan los mismos índices, aplicamos el método `reset_index()` para que los índices comiencen a partir de 0 y sean consecutivos, también colocamos el argumento `drop=True` para asegurarnos de borrar el primer índice

In [10]:
# reinicia los índices de los targets
target_valid_1 = target_valid_1.reset_index(drop=True)
target_valid_2 = target_valid_2.reset_index(drop=True)
target_valid_3 = target_valid_3.reset_index(drop=True)

# reinicia los índices de las predicciones
predicted_valid_1 = predicted_valid_1.reset_index(drop=True)
predicted_valid_2 = predicted_valid_2.reset_index(drop=True)
predicted_valid_3 = predicted_valid_3.reset_index(drop=True)

### 2.1. Volumen medio predicho y RMSE

Se va a calculas el volumen medio predicho y el RMSE del modelo para poder hacer un análisis de resultados del modelo y ver la calidad del modelo.

Para esto se creará una función que tomará como argumentos los valores correctos y las predicciones para cada data set que se realizaron en el punto anterior. Para calcular el RMSE se hará uso de la función `mean_squared_error`.

In [11]:
# Función para calcular el rmse y el volumen medio predicho
def rmse_cal(target_valid, predicted_valid):
    # Calcula el mse
    mse = mean_squared_error(target_valid, predicted_valid)
    # Saca la raíz al mse
    rmse = mse ** 0.5
    
    # calcula la media de la cantidad de barriles de las predicciones
    product_mean = predicted_valid.mean()
    
    print("Volumen medio de reservas predicho:", product_mean)
    print("RMSE del modelo:", rmse)
    return rmse, product_mean

In [12]:
# Llamada a la función
rmse_1, product_mean_1 = rmse_cal(target_valid_1, predicted_valid_1)
rmse_2, product_mean_2 = rmse_cal(target_valid_2, predicted_valid_2)
rmse_3, product_mean_3 = rmse_cal(target_valid_3, predicted_valid_3)

Volumen medio de reservas predicho: 92.59256778438035
RMSE del modelo: 37.5794217150813
Volumen medio de reservas predicho: 68.728546895446
RMSE del modelo: 0.893099286775617
Volumen medio de reservas predicho: 94.96504596800489
RMSE del modelo: 40.02970873393434


Sabemos que el volumen de reservas del pozo está expresado en miles de barriles

Conclusiones:

* La región 1 tiene un volumen promedio predicho de 92.59 mil barriles pero, en promedio, las predicciones erraron por 37.57 mil barriles.
* La región 2 tiene un volumen promedio predicho de 68.72 mil barriles pero, en promedio, las predicciones erraron por 893 barriles.
* La región 3 tiene un volumen promedio predicho de 94.96 mil barriles pero, en promedio, las predicciones erraron por 40.02 mil barriles.

La región con el mejor modelo de regresión lineal es la región dos, sin embargo, también es la región con menos volumen promedio predicho.

## 3. Inversión de 100 millones en 200 pozos

Con esa inversión, de media un pozo petrolífero debe producir al menos un valor de 500,000 dólares en unidades para evitar pérdidas, es decir, 111.1 barriles. Está cantidad se va a comparar con la cantidad media real de reservas en cada región con el fin de tener una idea de qué región se acerca más a está cantidad. Para esto se ocupará el `target_valid` de cada región.

In [13]:
# Media real de cada reserva
target_mean_1 = target_valid_1.mean()
target_mean_2 = target_valid_2.mean()
target_mean_3 = target_valid_3.mean()

# Imprimir los valores
print('Cantidad media de reservas en la región 1:', target_mean_1)
print('Cantidad media de reservas en la región 2:', target_mean_2)
print('Cantidad media de reservas en la región 3:', target_mean_3)

Cantidad media de reservas en la región 1: 92.07859674082927
Cantidad media de reservas en la región 2: 68.72313602435997
Cantidad media de reservas en la región 3: 94.88423280885438


Conclusiones:

La región que más se acerca es la región número 3, la cuál tiene una media real de 94.88 miles barriles de, sin embargo aún falta para los 111.1 miles de barriles necesarios.

Este cálculo abarca los 100000 pozos que tiene cada región. Sólo necesitamos los 200 mejores de cada región y calcular la ganancia que tienen. A continuación se realizará el cálculo de dicho beneficio.

### 3.1. Calculo de la ganancia

Cómo ya se mencionó, se van a calcular los beneficios de los mejores 200 pozos de cada región.

Esto se logrará creando una función que primero oredene las predicciones desde la mayor cantidad de barriles a la menor, después se seleccionarán sólo los mejores 200 pozos cuya predicción de unidades sea más grande. De esos 200 mejores pozos se calculará el beneficio de cada uno. Para esto se sabe que el ingreso de una unidad de producto es de 4500 dólares, es decir, 4500 dólares por cada mil barriles.

La función tomará por argumentos el `target` de cada región, las predicciones y la cantidad de pozos que recibirán la inversión. Y va a retornar la ganancia y las lista de predicciones de los mejores 200 pozos.

In [14]:
# Función para calcular las ganancias

def revenue(target, predicted, count):
    # Ordena los valores predichos en orden descendente
    predicted_sorted = predicted.sort_values(ascending=False)
    
    # Se guardan las 200 mejores predicciones
    top_200_predicted = predicted_sorted[:count]
    
    # Selecciona el volumne objetivo de los 200 pozos principales
    selected = target.iloc[predicted_sorted.index][:count]
    
    # retorna la lista de las mejores 200 predicciones y el calculo de la ganancia potencial
    return top_200_predicted, 4500 * selected.sum()  # Cada unidad tiene una ganancia de 4500 dólares
    

In [15]:
count = 200  # Sólo 200 pozos

# Llamar a la función y guarda la lista de los 200 mejores y la ganancia
top_200_predicted_1, revenue_1 = revenue(target_valid_1, predicted_valid_1, count)
top_200_predicted_2, revenue_2 = revenue(target_valid_2, predicted_valid_2, count)
top_200_predicted_3, revenue_3 = revenue(target_valid_3, predicted_valid_3, count)

# Imprime las ganancias potenciales de los mejores 200 pozos
print('La ganancia potencial de los mejores 200 pozos de la región 1 es:', revenue_1, 'dólares')
print('La ganancia potencial de los mejores 200 pozos de la región 2 es:', revenue_2, 'dólares')
print('La ganancia potencial de los mejores 200 pozos de la región 3 es:', revenue_3, 'dólares')

La ganancia potencial de los mejores 200 pozos de la región 1 es: 133208260.43139851 dólares
La ganancia potencial de los mejores 200 pozos de la región 2 es: 124150866.96681511 dólares
La ganancia potencial de los mejores 200 pozos de la región 3 es: 127103499.63599832 dólares


Conclusiones:

La inversión será de 100 millones de dólares. La región 1 es la que tiene un mayor margen de ganancias con las unidades predichas. Ya que tiene una ganancia de 133 millones de dólares, 33 millones de dólares que sobrepasan la inversión inicial.

La región 1 es la mejor opción para el desarrollo de pozos.

### 3.2. Riesgos y ganancias en cada región

Para esto se utilizará la técnica de bootstraping. Con esta técnica se puede conseguir un valor deseado, en nuestro caso será el beneficio predicho, se crean submuestras del conjunto de datos. Posteriormente se calculará la media de cada subconjunto. De este modo podemos obtener varios valores y estimar su distribución.

Para lograr lo anterior se va a crear una función que tomará por parámetro las predicciones (`predicted`) de los mejores 200 pozos de cada región.

En la función se va a declarar una lista vacía dónde se guardarán el cálculo de ganancias con las predicciones.

Se va a crear un ciclo for para crear 1000 submuestras de las predicciones de los 200 mejores pozos. Serán esas submuestras las que serán utilizadas para calcular la ganancias solo de los 200 mejores pozos y se irán almacenando en la lista vacía y así tener un conjunto de valores.

Además habrá un contador el cúal irá en aumento cuando la ganancia sea menor de 100 millones dolares para, posteriormente poder calcular el riesgo de perdidas

In [16]:
# Función para el bootstrapping
def calculate_bootstrapping(predicted):
    # Se especifica el randomState
    state = np.random.RandomState(12345)
    # se crea la lista vacía
    values = []
    # Contador que aumentara cuando el retorno sea negativo
    count = 0
    
    # Ciclo para crear 1000 muestras
    for i in range(1000):
        # Se crea una submuestra a partir de los conjuntos target y predicted
        predicted_subsample = predicted.sample(frac=1, replace=True, random_state=state)
        
        # Calcula la ganancia de las predicciones 
        revenue = 4500 * predicted_subsample.sum()
        
        # Sentencia if para saber si la ganancia es menor que la inversión
        if revenue < 100000000:
            count += 1
        
        values.append(revenue)
    
    values = pd.Series(values)  # Por conveniencia se transforma en un Series
    return values, count

Ya se tiene la función que realiza el trabajo de bootstrappping, ahora tenemos que llamarla para cada región con su respectivo `targe_valid` y `predicted_valid`. y guardar sus ganancias en variables.

In [17]:
# Guarda los valores de las ganancias de cada submuestra
revenues_1, count_1 = calculate_bootstrapping(top_200_predicted_1)
revenues_2, count_2 = calculate_bootstrapping(top_200_predicted_2)
revenues_3, count_3 = calculate_bootstrapping(top_200_predicted_3)

#### 3.2.1. Beneficio promedio

Se va a calcular el beneficio promedio de cada región

In [18]:
# Calculo del proemdio de los beneficios
revenue_mean_1 = revenues_1.mean()
revenue_mean_2 = revenues_2.mean()
revenue_mean_3 = revenues_3.mean()

# Imprime los valores
print('El beneficio promedio de la región 1 es de:', revenue_mean_1, 'dólares')
print('El beneficio promedio de la región 2 es de:', revenue_mean_2, 'dólares')
print('El beneficio promedio de la región 3 es de:', revenue_mean_3, 'dólares')

El beneficio promedio de la región 1 es de: 139944044.81195655 dólares
El beneficio promedio de la región 2 es de: 124856341.33227411 dólares
El beneficio promedio de la región 3 es de: 133203436.83229983 dólares


Conclusiones:

La región 1 es la que presenta una ganancia mayor, con 139 millones de dólares. Esto deja un margen de 39 millones después de la inversión inicial.

#### 3.2.2. Intervalo de confianza del 95%

Se va a calcular el intervalo de confianza del 95%, para esto debemos tomar el 2.5% tanto del extremo inferior como del extremo superior.

In [19]:
# Cuantil del 2.5%
lower_1 = revenues_1.quantile(0.025)
lower_2 = revenues_2.quantile(0.025)
lower_3 = revenues_3.quantile(0.025)

# Cuantil del 97.5%
upper_1 = revenues_1.quantile(0.975)
upper_2 = revenues_3.quantile(0.975)
upper_3 = revenues_3.quantile(0.975)

# Impresion del intervalo
print(f'El intervalo del 95% de la region 1 es de entre {lower_1} dólares y {upper_1} dólares')
print(f'El intervalo del 95% de la region 2 es de entre {lower_2} dólares y {upper_2} dólares')
print(f'El intervalo del 95% de la region 3 es de entre {lower_3} dólares y {upper_3} dólares')

El intervalo del 95% de la region 1 es de entre 139104578.9235707 dólares y 140792451.05654183 dólares
El intervalo del 95% de la region 2 es de entre 124818499.87004468 dólares y 133937189.00814931 dólares
El intervalo del 95% de la region 3 es de entre 132503826.13157943 dólares y 133937189.00814931 dólares


Conclusiones:

Podemos decir con un 95 % de probabilidad que la región 1, tendrá ganancias de entre 139 millones y 140 millones de dólares. Esta es la mejor región para invertir.

#### 3.2.3. Riesgo de pérdidas

Con la variable `count` que está guardada para región podemos calcular el riesgo de pérdidas. Esta variable tiene guardado la cantidad de retornos negativos, esto se analizará contra el total de muestras de bootstrap (1000).

In [20]:
# Calculo del riesgo de pérdida en las tres regiones
risk_1 = (count_1 * 100) / 1000
risk_2 = (count_1 * 100) / 1000
risk_3 = (count_1 * 100) / 1000

# Imprime el riesgo de cada región
print('El riesgo de pérdida en la región 1 es de:', risk_1)
print('El riesgo de pérdida en la región 2 es de:', risk_2)
print('El riesgo de pérdida en la región 3 es de:', risk_3)

El riesgo de pérdida en la región 1 es de: 0.0
El riesgo de pérdida en la región 2 es de: 0.0
El riesgo de pérdida en la región 3 es de: 0.0


Conclusiones:

Con nuestras predicciones para los 200 mejores pozos, se calcula que el riesgo de pérdida es de 0%, por lo que se puede invertir en cualquiera de las tres regiones

### 3.3. Conclusiones


Haciendo un cálculo de ganancia de la predicciones que salieron del modelo en cada región, la **región 1** es la que tiene un margen mayor de ganancias.

Con el método de bootstrapping, la **región 1** es la región que tuvo mayor beneficio promedio (139 milllones de dólares), la que tiene el intervalo de confianza de 95% más grande (de 139 a 140 millones de dólares) y no hay riesgo de pérdida. Por lo que la región 1 es la mejor para onvertir los 100 millones de dólares